datasets used  

[autocorrect](https://www.kaggle.com/datasets/kaggleqrdl/autocorrect) 

[contractions==0.1.72](https://pypi.org/project/contractions/0.1.72/)

[Feedback Prize - Predicting Effective Arguments](https://www.kaggle.com/competitions/feedback-prize-effectiveness)

[Feedback Prize - Evaluating Student Writing](https://www.kaggle.com/competitions/feedback-prize-2021)

[reference notebook](https://www.kaggle.com/code/shigengtian/prepare-mlm-data)

In [1]:
%%capture

!pip install ../input/pycontractions/results/anyascii-0.3.1-py3-none-any.whl
!pip install ../input/pycontractions/results/pyahocorasick-1.4.4-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install ../input/pycontractions/results/textsearch-0.0.21-py2.py3-none-any.whl
!pip install ../input/pycontractions/results/contractions-0.1.72-py2.py3-none-any.whl
!pip install -q "../input/autocorrect/autocorrect-2.6.1.tar"

import contractions
import pandas as pd
import os
import re
import codecs
from typing import Dict, List, Tuple
from text_unidecode import unidecode


In [2]:
input_path_2021 = '../input/feedback-prize-2021/'
input_path_2022 = '../input/feedback-prize-effectiveness/'

In [3]:
# train_2021=pd.read_csv(input_path_2021 + 'train.csv')
train_2022=pd.read_csv(input_path_2022 + 'train.csv')
train_2021 = pd.read_csv('../input/deberta-v3-large-feedback-pseudo/pseudo616.csv')

In [4]:
# https://www.kaggle.com/code/brandonhu0215/feedback-deberta-large-lb0-619

def replace_encoding_with_utf8(error: UnicodeError) -> Tuple[bytes, int]:
    return error.object[error.start : error.end].encode("utf-8"), error.end


def replace_decoding_with_cp1252(error: UnicodeError) -> Tuple[str, int]:
    return error.object[error.start : error.end].decode("cp1252"), error.end

codecs.register_error("replace_encoding_with_utf8", replace_encoding_with_utf8)
codecs.register_error("replace_decoding_with_cp1252", replace_decoding_with_cp1252)


def resolve_encodings_and_normalize(text: str) -> str:
    text = (
        text.encode("raw_unicode_escape")
        .decode("utf-8", errors="replace_decoding_with_cp1252")
        .encode("cp1252", errors="replace_encoding_with_utf8")
        .decode("utf-8", errors="replace_decoding_with_cp1252")
    )
    
    text = unidecode(text)
    
    return text


def fetch_essay(essay_id: str, txt_dir: str):
    essay_path = os.path.join(input_path_2022 + txt_dir, essay_id + '.txt')
    # print(input_path_2022 + txt_dir, essay_id + '.txt')
    essay_text = open(essay_path, 'r').read()
    
    return essay_text

In [5]:
# df = test_origin.copy()
# SEP = CFG.tokenizer.sep_token

train_2022['discourse_text'] = train_2022['discourse_text'].apply(resolve_encodings_and_normalize)
train_2022['essay_text'] = train_2022['essay_id'].transform(fetch_essay, txt_dir='train')
train_2022['essay_text'] = train_2022['essay_text'].apply(resolve_encodings_and_normalize)
# train_2022['text'] = train_2022['discourse_type'] + ' ' + train_2022['discourse_text'] + '[SEP]' + train_2022['essay_text']

train_2022['text'] = train_2022['discourse_type'].str.lower() + ' ' + train_2022['discourse_text'].str.lower() + ' ' + train_2022['essay_text'].str.lower()

In [6]:
# train_2021['discourse_text'] = train_2021['discourse_text'].apply(resolve_encodings_and_normalize)
# train_2021['essay_text'] = train_2021['id'].transform(fetch_essay, txt_dir='train')
# train_2021['essay_text'] = train_2021['essay_text'].apply(resolve_encodings_and_normalize)


train_2021['text'] = train_2021['discourse_type'].str.lower() + ' ' + train_2021['discourse_text'].str.lower() + ' ' + train_2021['essay_text'].str.lower()

In [7]:
train_2022['text'] = train_2022['text']
train_2021['text'] = train_2021['text']

In [8]:
df1 = train_2022[['text']]
df2 = train_2021[['text']]

In [9]:
text_df  = pd.concat([df1, df2]).drop_duplicates()
text_df.head()

,text
0,"lead hi, i'm isaac, i'm going to be writing ab..."
1,"position on my perspective, i think that the f..."
2,claim i think that the face is a natural landf...
3,"evidence if life was on mars, we would know by..."
4,counterclaim people thought that the face was ...


In [10]:
text_df = text_df.reset_index(drop=True)
print(text_df)
print("\n\n\n")
text_df.text[0]

                                                     text
0       lead hi, i'm isaac, i'm going to be writing ab...
1       position on my perspective, i think that the f...
2       claim i think that the face is a natural landf...
3       evidence if life was on mars, we would know by...
4       counterclaim people thought that the face was ...
...                                                   ...
144261  claim  misdeeds are when the advice-giver is p...
144262  evidence an example of this is when you ask ge...
144263  counterclaim now, i know what you probably say...
144264  rebuttal  what are the odds that seven of your...
144265  concluding statement the odds are in your favo...

[144266 rows x 1 columns]






'lead hi, i\'m isaac, i\'m going to be writing about how this face on mars is a natural landform or if there is life on mars that made it. the story is about how nasa took a picture of mars and a face was seen on the planet. nasa doesn\'t know if the landform was created by life on mars, or if it is just a natural landform.  hi, i\'m isaac, i\'m going to be writing about how this face on mars is a natural landform or if there is life on mars that made it. the story is about how nasa took a picture of mars and a face was seen on the planet. nasa doesn\'t know if the landform was created by life on mars, or if it is just a natural landform. on my perspective, i think that the face is a natural landform because i dont think that there is any life on mars. in these next few paragraphs, i\'ll be talking about how i think that is is a natural landform\n\ni think that the face is a natural landform because there is no life on mars that we have descovered yet. if life was on mars, we would kno

In [11]:

from autocorrect import Speller
import nltk
from nltk.corpus import brown,words


def autospell(text):
    spell = Speller(lang='en', fast=False)
    spells = [spell(w) for w in (nltk.word_tokenize(text))]
    return " ".join(spells) 

def __concat(object1, object2):
    unicode = str
    if isinstance(object1, str) or isinstance(object1, unicode):
        object1 = [object1]
    if isinstance(object2, str) or isinstance(object2, unicode):
        object2 = [object2]
    return object1 + object2


def __capitalize_first_char(word):
    return word[0].upper() + word[1:]


def split(word, language='en'):
    
    dictionary = set(words.words())
    max_index = len(word)
    for index, char in enumerate(word):
        left_compound = word[0:max_index-index]
        right_compound_1 = word[max_index-index:max_index]
        right_compound_2 = word[max_index-index+1:max_index]
        if right_compound_1:
            right_compound1_upper = right_compound_1[0].isupper()
        if right_compound_2:
            right_compound2_upper = right_compound_2[0].isupper()
      
        if index > 0 and len(left_compound) > 1 and not left_compound in dictionary:
            left_compound = __capitalize_first_char(left_compound)
        is_left_compound_valid_word = len(left_compound) > 1 and left_compound in dictionary
        if is_left_compound_valid_word and \
                ((not split(right_compound_1, language) == '' and not right_compound1_upper) \
                or right_compound_1 == ''):
            return [compound for compound in __concat(left_compound, split(right_compound_1, language))\
                    if not compound == '']
        elif is_left_compound_valid_word and word[max_index-index:max_index-index+1] == 's' and \
            ((not split(right_compound_2, language) == '' and not right_compound2_upper) \
            or right_compound_2 == ''):
            return [compound for compound in __concat(left_compound, split(right_compound_2, language))\
                    if not compound == '']
  
    if not word == '' and word in dictionary:
        return [word]
    
    elif not word == '' and __capitalize_first_char(word) in dictionary:

        return [__capitalize_first_char(word)]
    else:
        return ""
def fixed_word(words):
    words = words.lower() #without it the previous algo implementation didn't work for upper case char in middle
    
    expanded_words = []   
    for word in words.split():
        expanded_words.append(contractions.fix(word))  
    words = ' '.join(expanded_words)
    expanded_words = []
    for word in words.split():
        if("’" in word):
            expanded_words.append([word])
            continue
        if("sep" == word):
            expanded_words.append([word])
            continue
        output = split(word)
        if(output == ""):
            output = autospell(word)
            output = [output]
        expanded_words.append(output)
        
      

    text = ' '.join(' '.join(l) for l in expanded_words) 

    return text



In [12]:
# %%time

# # for i in range(len(text_df)):
# #     text_df.text[i] = fixed_word(text_df.text[i])

In [13]:
# from spellchecker import SpellChecker

# spell = SpellChecker()

# # find those words that may be misspelled
# misspelled = spell.unknown(['something', 'is', 'hapenning', 'artfactory'])

# for word in misspelled:
#     # Get the one `most likely` answer
#     print(spell.correction(word))

#     # Get a list of `likely` options
#     print(spell.candidates(word))

In [14]:
text = fixed_word("i'm somethng of a paintr myself") 
text

'i am something of a paint r myself'

In [15]:
# !pip download Caribe
# !pip install Caribe

In [16]:

# import Caribe as cb
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import pandas as pd

# sentence = 'The smelt of sep fliwers bring back memories.'
# standard = cb.phrase_decode(sentence)
# standard = cb.trinidad_decode(standard)
# fixed = cb.caribe_corrector(standard)
# print(fixed)



tokenizer1 = AutoTokenizer.from_pretrained("../input/spellcheckers/t5-base-spellchecker")
model1 = AutoModelForSeq2SeqLM.from_pretrained("../input/spellcheckers/t5-base-spellchecker")
max_length1 = 256

tokenizer2 = AutoTokenizer.from_pretrained("../input/spellcheckers/spelling-correction-english-base")
model2 = AutoModelForSeq2SeqLM.from_pretrained("../input/spellcheckers/spelling-correction-english-base")
max_length2 = 2048


In [17]:
%%time
import torch
import gc
import re
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

def correct(inputs,max_length,tokenizer,model):
    sentences = re.split(r'(?<=\w\.)\s', inputs)
    model = model.to(device)
    for i in range(len(sentences)):
        expanded_words = [] 
        for word in sentences[i].split():
            expanded_words.append(contractions.fix(word))
        sentences[i] = " ".join(expanded_words)
        input_ids = tokenizer.encode(sentences[i],return_tensors='pt').to(device)
        sample_output = model.generate(
            input_ids,
            do_sample=False,
            num_beams=50, 
            max_length=max_length,
            top_p=0.99,
            num_return_sequences=1
        )
        sentences[i] = tokenizer.decode(sample_output[0], skip_special_tokens=True)
        sentences[i] += '.'
    del model
    torch.cuda.empty_cache()
    gc.collect()
    return " ".join(sentences)


cpu
CPU times: user 336 µs, sys: 0 ns, total: 336 µs
Wall time: 350 µs


In [18]:
# %%time
# text_df1 = text_df.copy()
# text_df.text = text_df['text'].apply(lambda x : correct(x,max_length1,tokenizer1,model1))
# text

In [19]:
%%time

for i in range(len(text_df)):
    expanded_words = [] 
    for word in text_df.text[i].split():
        expanded_words.append(contractions.fix(word))
    text_df.text[i] = " ".join(expanded_words)

CPU times: user 11min 39s, sys: 276 ms, total: 11min 40s
Wall time: 11min 39s


In [20]:
test = text_df.sample(frac = 0.2)
 
# Creating dataframe with
# rest of the 50% values
train = text_df.drop(test.index)

train.to_csv('train.txt', header=False, index=False)
test.to_csv('test.txt', header=False, index=False)

len(train),len(test)

(115413, 28853)

In [21]:
# %%time

# text_df1.text = text_df1['text'].apply(lambda x : correct(x,max_length2,tokenizer2,model2))
# text

In [22]:
# test = text_df1.sample(frac = 0.2)
 
# # Creating dataframe with
# # rest of the 50% values
# train = text_df1.drop(test.index)

# train.to_csv('train1.txt', header=False, index=False)
# test.to_csv('test1.txt', header=False, index=False)

# len(train),len(test)

# train tokenizer

In [23]:
def get_training_corpus():
    dataset = text_df["text"]
    for start_idx in range(0, len(dataset), 1000):
        samples = dataset[start_idx : start_idx + 1000]
        yield samples
training_corpus = get_training_corpus()


In [24]:
training_corpus

<generator object get_training_corpus at 0x7fbbbb1c1dd0>

In [25]:
from transformers import AutoTokenizer
from transformers.models.deberta_v2 import DebertaV2TokenizerFast

old_tokenizer = DebertaV2TokenizerFast.from_pretrained("microsoft/deberta-v3-large")
print(old_tokenizer.is_fast)

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/580 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.7/site-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


True


In [26]:
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)
print(tokenizer.is_fast)



True


In [27]:
tokenizer.save_pretrained("debertav3new-tokenizer")

('debertav3new-tokenizer/tokenizer_config.json',
 'debertav3new-tokenizer/special_tokens_map.json',
 'debertav3new-tokenizer/tokenizer.json')